In [1]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk.tag import pos_tag
import json
from collections import Counter
from nltk.util import ngrams


pd.set_option('display.width', 10000)

In [2]:
stopword = nltk.corpus.stopwords.words('english')
#stopword.extend(['best', 'picture', 'musical', 'Best', 'Picture', 'Musical', 'Comedy', 'comedy', 'RT'])

In [3]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [4]:
copy = gg.copy(deep=True)

In [5]:
def removePunc(x):
#     new_string = x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\":()]+', '', x) #taking out punctuation i.e ? ! . ' : ( ) and " 
    x = re.sub(r'(RT|rt) ', '', x) #taking out the initial "RT "
    x= re.sub(r'(g|G)olden (g|G)lobes', '', x)
    return x.strip()
#copy['text'] = copy['text'].str.lower()
#copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))


In [10]:
def getMovieNom(winner,tweets):
        f=open('MovieDatabase.txt',encoding='utf-16')
        moviedic={}
        tweets = tweets.str.lower()
        tweets = tweets.apply(lambda x: removePunc(str(x)))
        for i in f.read().split('\n'):
            try:
                k=i
                if k:
                    moviedic[k.lower()]=1
            except:
                continue
        f.close()
        count=0
        movies = {}
        movies[winner]=1
        for tweet in tweets:
            if winner in tweet:
                token = nltk.word_tokenize(tweet)
                bigrams = ngrams(token,2)
                trigrams= ngrams(token,3)
                GG=Counter(bigrams)+Counter(trigrams)+Counter(token)
                for i in GG.most_common(50):
                    if len(i[0])>3:
                        xx=i[0]
                    else:
                        xx=' '.join(k for k in i[0]).lower()
                    if 'the' in xx and len(xx.split())<3:
                        continue
                    if xx in moviedic.keys():
                        movies[xx]=1
                        count+=1
            if count>10:
                break
        a=list(movies.keys())[0:6]
        if winner in a:
            a.remove(winner)
        return a

In [7]:
def Achive_names(tweet,winner,threshhold):
    blacklist=['golden','best','globe','motion','actor','actress','hero','picture','drama']
    count=0
    names={}
    nltk_results = nltk.ne_chunk(pos_tag(nltk.word_tokenize(tweet)))
    for chunk in nltk_results:
        if type(chunk) == nltk.tree.Tree:
            if chunk.label() == 'PERSON':
                l=[]
                for c in chunk[0:4]:
                    if c[0].lower() in blacklist:
                        break
                    else:
                        l.append(c[0])
                temp = ' '.join(l)
                if len(temp.split())==2:
                    if temp not in names:
                        count+=1
                        names[temp.lower()]=1
                    else:
                        names[temp.lower()]+=1
    if count>=threshhold:
        return names
    return []

In [8]:
def getNominee(award,winner,tweets):
        res=[]
        if 'actor'in award or 'actress'in award or 'performance' in award or 'director'in award or'award'in award:
            for tweet in tweets:
                if winner in tweet.lower():
                    result=Achive_names(tweet,winner,5)
                    if result:
                        candidate=list(result.keys())
                        if winner in candidate:
                            candidate.remove(winner)
                            res.extend(candidate)
            return list(set(res))
        else:
            return getMovieNom(winner,tweets)
        


In [11]:
getNominee("best original score - motion picture","life of pi",copy['text'])

['lincoln', 'argo', 'the hunger games', 'get out']